## Notebook Principal

Contém os testes com a árvore trie, além de exemplos de compressão e decompressão

In [ ]:
from trie import RadixTree
from lzw import *
from functions import *
import pandas as pd
import time

# Função para escrever os códigos criados num arquivo de texto

# Script de testes

1. Compressão:
    * Codificação de textos
    * Criação de lista de códigos
    * Compressão dos códigos em binário
    * Comparação do tamanho dos textos com os códigos binários
    * Taxa de compressão
    * Tempo de compressão

2. Decompressão
    * Decompressão dos códigos binários
    * Decodificação dos códigos
    * Comparação do texto decodificado com o arquivo original
    * Tempo de decompressão

In [2]:
def compress_data(path, files):
    df = pd.DataFrame(
        columns=[
            "Arquivo",
            "Formato",
            "Tamanho original (bits)",
            "Tamanho comprimido (bits)",
            "Tamanho Dicionário",
            "Qtd de bits por código",
            "Taxa de compressão",
            "Tempo de compressão (ms)",
            "Tempo Relativo (ms/kB)",
        ]
    )
    for file_name in files:
        f = path + "/" + file_name
        with open(f, "r", encoding="utf-8") as file:
            s = file.read()
            file_size = len(s)
            print("-" * 50)
            print(f"Arquivo de entrada: {file_name}")
            print(f"Tamanho: {len(s)} bytes")

            for format in ["ascii", "binary"]:
                print("-" * 50)
                print(f"Encoding {format} text...")
                if format == "binary":
                    s = str_to_bin(s)

                # Repete 20 vezes
                n_loops = 20
                for i in range(n_loops):
                    print(f"Encode start: ({i+1}/{n_loops})")
                    # Codificação
                    start_time = time.monotonic_ns()
                    encoder = LZW_Encoder(by=format)
                    encoded = encoder.encode(s)
                    compressed, n_bits = compress(encoded)
                    end_time = time.monotonic_ns()
                    # Fim da codificação

                    dictionary = encoder.dictionary
                    dict_size = len(dictionary)
                    compressed_size = len(compressed)
                    compressed_rate = file_size / compressed_size
                    code_time = (end_time - start_time) / 1_000_000
                    rel_time = code_time / file_size
                    print(f"Encode end: {code_time} ms")

                    # Dataframe
                    df.loc[len(df)] = [
                        file_name,  # Nome do arquivo
                        format,  # 'ascii' ou 'binary'
                        8 * file_size,  # Tamanho em bits
                        len(compressed),  # Tamanho em bits
                        dict_size,  # Número de strings
                        n_bits,  # Bits por código
                        8 * compressed_rate,  # Taxa de compressão
                        code_time,  # Tempo em ms
                        1024 * rel_time,  # Tempo relativo em ms/kB
                    ]
    return df

In [3]:
def main():
    """
    Exemplo de uso.
    """
    # Arquivos de entrada
    n_files = 4
    path = "tests"
    files = [f"ex ({i}).txt" for i in range(1, n_files + 1)]  # "ex (i).txt"
    files.append("functions.py")
    files.append("lzw.py")
    files.append("trie.py")
    # Criação do Dataframe
    df = compress_data(path, files)
    print(df)
    df.to_csv("out.csv", index=False)

In [4]:
if __name__ == "__main__":
    main()

--------------------------------------------------
Arquivo de entrada: ex (1).txt
Tamanho: 18 bytes
--------------------------------------------------
Encoding ascii text...
Encode start: (1/20)
Encode end: 0.587453 ms
Encode start: (2/20)
Encode end: 1.290226 ms
Encode start: (3/20)
Encode end: 0.310186 ms
Encode start: (4/20)
Encode end: 0.352926 ms
Encode start: (5/20)
Encode end: 0.294827 ms
Encode start: (6/20)
Encode end: 0.291667 ms
Encode start: (7/20)
Encode end: 0.334367 ms
Encode start: (8/20)
Encode end: 0.402415 ms
Encode start: (9/20)
Encode end: 0.294237 ms
Encode start: (10/20)
Encode end: 0.292947 ms
Encode start: (11/20)
Encode end: 0.293216 ms
Encode start: (12/20)
Encode end: 0.294666 ms
Encode start: (13/20)
Encode end: 0.300067 ms
Encode start: (14/20)
Encode end: 0.383265 ms
Encode start: (15/20)
Encode end: 0.402776 ms
Encode start: (16/20)
Encode end: 0.290997 ms
Encode start: (17/20)
Encode end: 0.322936 ms
Encode start: (18/20)
Encode end: 0.294207 ms
Encode 

# Testes de Decodificação

In [ ]:
files = ["ex (3).txt", "functions.py", "trie.py"]
encoder = LZW_Encoder()
decoder = LZW_Decoder()
for file_name in files:
    with open("tests/" + file_name, "r", encoding="utf-8") as file:
        s = file.read()
        encoded = encoder.encode(s)
        compressed, n_bits = compress(encoded)
        decoded = decompress(compressed, n_bits)
        text = decoder.decode(decoded)
        print("-" * 50)
        print(file_name)
        print("-" * 50)
        print(text)
        print("\n")

--------------------------------------------------
ex (3).txt
--------------------------------------------------
UNIVERSIDADE FEDERAL DE MINAS GERAIS
Instituto de Ciências Exatas
Departamento de Ciência da Computação

DCC207 -- Algoritmos 2
Prof. Renato Vimieiro

Trabalho Prático 1 -- Manipulação de sequências

Objetivos

Nesse trabalho serão abordados os aspectos práticos de manipulação de sequências. Especificamente, serão explorados aspectos de implementação de árvores de prefixo aplicada ao problema de compressão de arquivos.

O objetivo secundário é fixar o conteúdo. Entende-se que ao implementar a estrutura o aluno conseguirá compreender melhor os conceitos explorados. Dessa forma, o conteúdo teórico será melhor absorvido e fixado. Além disso, os alunos terão a oportunidade de ver conceitos não abordados na disciplina, no caso específico, um método de compressão de arquivos.

Tarefas

Os alunos deverão implementar um algoritmo para resolver o problema de 
compressão de arquivos a